In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os
from PIL import ImageFont, ImageDraw, Image
import sys

actions = ['j','u','p','pp']
seq_length = 30
secs_for_action = 10

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

while cap.isOpened():
    for idx, action in enumerate(actions):
        data = []

        ret, img = cap.read()
        img = cv2.flip(img, 1) # 좌우 반전

        img = np.full(shape=(512,512,3),fill_value=255,dtype=np.uint8)
        img = Image.fromarray(img) #img배열을 PIL이 처리가능하게 변환

        draw = ImageDraw.Draw(img)
        # myfont = ImageFont.truetype("C:/Users/kelly/IdeaProjects/Project mid/font/MaruBuri-Bold.ttf",20)
        org=(10,30)# 글자 표시할 위치

#        draw.text(org, f'Waiting for collecting {action.upper()} action...', font=myfont, fill=(0,0,0)) # 추측으론 fill은 글자색 0,0,0은 검은색
#        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
#        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=font, fontScale=1, color=(255, 255, 255), thickness=2)
        img = np.array(img) # numpy가 처리할 수 있도록 다시 변환
        cv2.imshow('img', img)
        cv2.waitKey(1000)

        start_time = time.time()

        if img is None:
            print('Image load failed')
            sys.exit()

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                        
                    check = joint[5][0] - joint[17][0]
                    point = joint[5][0] - joint[0][0]
                    
                    
                    comp_tip_1 = joint[[4,8,12,16], 0:1]
                    comp_tip_2 = joint[[8,12,16,20], 0:1]
                                       
                    tip_to_tip = comp_tip_1 - comp_tip_2
                    
                    tip_to_tip = tip_to_tip / np.linalg.norm(tip_to_tip)
                                       
                                        
                    zero = joint[[0,0,0,0,0], 1:2 ]
                    tip = joint[[4,8,12,16,20], 1:2]
                    
                    zero_to_tip = zero - tip
                    
                    zero_to_tip = zero_to_tip / np.linalg.norm(zero_to_tip)
                    
                    print(zero_to_tip)
                    print(tip_to_tip)

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    
                    v = v2 - v1 # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                    angle = np.degrees(angle) # Convert radian to degree

                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)
                                            
                                            
                    
                    d = np.append(v.flatten(), check)
                    d = np.append(d, point)
                    d = np.append(d,zero_to_tip)
                    d = np.append(d,tip_to_tip)
                    
                    
                    #d = np.concatenate([joint.flatten(), d])
                    d = np.concatenate([d, angle_label])

                    #d = np.concatenate([joint.flatten(), angle_label])
                    #d = np.concatenate([d, check])
                    
                

                    data.append(d)

                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

        # Create sequence data
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    break


[[ 0.206163  ]
 [ 0.72494573]
 [-0.1303516 ]
 [-0.40652037]
 [-0.49970007]]
[[-0.47209948]
 [ 0.10667633]
 [-0.78282559]
 [-0.39105796]]
[[ 0.34933823]
 [ 0.91577908]
 [ 0.09316168]
 [-0.05990885]
 [-0.16444849]]
[[ 0.13560583]
 [ 0.53077455]
 [-0.69160834]
 [-0.4707094 ]]
[[ 0.31115708]
 [ 0.93336298]
 [ 0.09005387]
 [-0.01065125]
 [-0.15424549]]
[[ 0.04812525]
 [ 0.68475523]
 [-0.38865797]
 [-0.61460492]]
[[ 0.09843866]
 [ 0.92797626]
 [-0.0314911 ]
 [-0.18812078]
 [-0.30461249]]
[[-0.70158407]
 [ 0.3424747 ]
 [-0.52921768]
 [-0.33229434]]
[[ 0.12485639]
 [ 0.95224797]
 [ 0.00731252]
 [-0.12224564]
 [-0.25027427]]
[[-0.76152809]
 [ 0.20835044]
 [-0.52647612]
 [-0.31541713]]
[[ 0.2024827 ]
 [ 0.96861066]
 [ 0.12106701]
 [ 0.03239898]
 [-0.07132481]]
[[-0.81197541]
 [ 0.3964556 ]
 [-0.38700834]
 [-0.18369388]]
[[ 0.20778706]
 [ 0.96553216]
 [ 0.12911283]
 [ 0.04198987]
 [-0.07835124]]
[[-0.79398698]
 [ 0.40890279]
 [-0.39844306]
 [-0.20886915]]
[[0.30850196]
 [0.91889932]
 [0.21619461]

[[0.19699344]
 [0.94765221]
 [0.21384405]
 [0.13187878]
 [0.00525139]]
[[-0.7251034 ]
 [ 0.66359314]
 [-0.18215005]
 [-0.02627854]]
[[0.40862656]
 [0.85597587]
 [0.2497787 ]
 [0.18115699]
 [0.07157082]]
[[-0.48413538]
 [ 0.84309161]
 [-0.22719453]
 [-0.05649884]]
[[0.4862781 ]
 [0.82152609]
 [0.23465327]
 [0.18156675]
 [0.02449173]]
[[-0.43998712]
 [ 0.87660153]
 [-0.13407356]
 [-0.14144037]]
[[0.3416362 ]
 [0.81876507]
 [0.36445116]
 [0.27068513]
 [0.08254319]]
[[-0.73216566]
 [ 0.6633719 ]
 [-0.10876344]
 [-0.1097346 ]]
[[0.26455214]
 [0.91012225]
 [0.27690201]
 [0.1577859 ]
 [0.01088736]]
[[-0.7550476 ]
 [ 0.64876853]
 [-0.08650529]
 [-0.03897886]]
[[0.31566965]
 [0.89097891]
 [0.27679503]
 [0.17271922]
 [0.00786369]]
[[-0.73915881]
 [ 0.67090324]
 [-0.05893424]
 [-0.00773653]]
[[0.31465951]
 [0.85794759]
 [0.37288494]
 [0.15284461]
 [0.05010666]]
[[-0.81252141]
 [ 0.58117386]
 [ 0.03581297]
 [ 0.02762851]]
[[ 0.05235193]
 [ 0.97974106]
 [ 0.12101352]
 [-0.06388584]
 [-0.13653227]]


[[-0.04763034]
 [ 0.66531768]
 [-0.42081794]
 [-0.32677131]
 [-0.5207845 ]]
[[ 0.53000609]
 [-0.18361775]
 [-0.81373661]
 [-0.15235091]]
[[-0.12045868]
 [ 0.6543049 ]
 [-0.39800975]
 [-0.34130777]
 [-0.531481  ]]
[[ 0.52308295]
 [-0.25703089]
 [-0.80101266]
 [-0.13674086]]
[[ 0.02060464]
 [ 0.71641549]
 [-0.35393114]
 [-0.31743049]
 [-0.51019108]]
[[ 0.57841028]
 [-0.24846658]
 [-0.75329216]
 [-0.19041225]]
[[ 0.01575243]
 [ 0.7193412 ]
 [-0.35385164]
 [-0.31106651]
 [-0.51022225]]
[[ 0.5745882 ]
 [-0.264032  ]
 [-0.74085184]
 [-0.22643774]]
[[ 0.03577925]
 [ 0.70011193]
 [-0.38099728]
 [-0.31954039]
 [-0.51117329]]
[[ 0.60337355]
 [-0.23666666]
 [-0.72125244]
 [-0.24438529]]
[[-0.00197053]
 [ 0.69675391]
 [-0.3738232 ]
 [-0.32644869]
 [-0.51789727]]
[[ 0.54189411]
 [-0.18891796]
 [-0.78680483]
 [-0.22715402]]
[[ 0.04185118]
 [ 0.75967544]
 [-0.33118458]
 [-0.29502633]
 [-0.47372772]]
[[ 0.51782134]
 [-0.11561759]
 [-0.82502761]
 [-0.19448155]]
[[ 0.03201265]
 [ 0.75227402]
 [-0.336712

[[0.33039149]
 [0.87614126]
 [0.29658177]
 [0.18763329]
 [0.00713916]]
[[-0.69060139]
 [ 0.72226698]
 [-0.03569261]
 [ 0.0112324 ]]
[[ 0.30022267]
 [ 0.89041955]
 [ 0.3001561 ]
 [ 0.161918  ]
 [-0.02661292]]
[[-0.6929896 ]
 [ 0.72025203]
 [-0.03151389]
 [ 0.00305073]]
[[ 3.03792516e-01]
 [ 8.87078229e-01]
 [ 2.96825694e-01]
 [ 1.80822616e-01]
 [-1.08234841e-04]]
[[-0.69135698]
 [ 0.72146339]
 [-0.03842131]
 [ 0.00631691]]
[[0.2978621 ]
 [0.88182458]
 [0.31087599]
 [0.19188771]
 [0.01409967]]
[[-0.69245504]
 [ 0.72061485]
 [-0.03337585]
 [ 0.01031117]]
[[0.29729685]
 [0.8756324 ]
 [0.31491549]
 [0.21031777]
 [0.03843369]]
[[-0.69136666]
 [ 0.72154398]
 [-0.03008197]
 [ 0.02194296]]
[[0.27922439]
 [0.88147955]
 [0.32555012]
 [0.19690181]
 [0.01656314]]
[[-0.68991826]
 [ 0.72276462]
 [-0.03959448]
 [ 0.00750783]]
[[ 0.25116345]
 [ 0.88990905]
 [ 0.32839385]
 [ 0.19265996]
 [-0.00429288]]
[[-0.69572735]
 [ 0.71737251]
 [-0.03573298]
 [ 0.00795531]]
[[ 0.2580967 ]
 [ 0.89521635]
 [ 0.318497

[[0.21992183]
 [0.71909938]
 [0.62672756]
 [0.18069934]
 [0.09534561]]
[[-0.61378846]
 [-0.13014936]
 [ 0.77648449]
 [ 0.0582813 ]]
[[0.21144582]
 [0.72171998]
 [0.62874581]
 [0.17470205]
 [0.09256799]]
[[-0.60693161]
 [-0.12467155]
 [ 0.78282202]
 [ 0.05727751]]
[[0.21229147]
 [0.71904262]
 [0.62979126]
 [0.17971917]
 [0.09473137]]
[[-0.60415923]
 [-0.12587082]
 [ 0.78463834]
 [ 0.05908329]]
[[0.20697392]
 [0.72079808]
 [0.63026642]
 [0.17712977]
 [0.09487471]]
[[-0.60440003]
 [-0.12851454]
 [ 0.78484226]
 [ 0.04698131]]
[[0.21597799]
 [0.72147189]
 [0.62810697]
 [0.17451062]
 [0.08865378]]
[[-0.61708499]
 [-0.12216383]
 [ 0.77476892]
 [ 0.06336584]]
[[0.21478252]
 [0.71724384]
 [0.63093403]
 [0.17636669]
 [0.10122637]]
[[-0.61794073]
 [-0.12493425]
 [ 0.77334543]
 [ 0.06691438]]
[[0.21366458]
 [0.71967484]
 [0.63270652]
 [0.16839169]
 [0.08799016]]
[[-0.6087219 ]
 [-0.12780803]
 [ 0.78153042]
 [ 0.0483007 ]]
[[0.214407  ]
 [0.71646443]
 [0.63604184]
 [0.16954864]
 [0.08609526]]
[[-0.

...

[[0.35062486]
 [0.83213088]
 [0.28649756]
 [0.18812736]
 [0.25912864]]
[[-0.58478654]
 [ 0.61037108]
 [ 0.05454219]
 [-0.53150447]]
[[0.3491556 ]
 [0.83418894]
 [0.2878397 ]
 [0.19545877]
 [0.24731226]]
[[-0.60306928]
 [ 0.61585983]
 [ 0.05154449]
 [-0.50434836]]
[[0.34927229]
 [0.83789933]
 [0.27981779]
 [0.18963275]
 [0.24834451]]
[[-0.59803446]
 [ 0.60913383]
 [ 0.06076633]
 [-0.51731829]]
[[0.35342266]
 [0.83723997]
 [0.28461831]
 [0.17622304]
 [0.24911746]]
[[-0.59637851]
 [ 0.59923761]
 [ 0.07583121]
 [-0.52867437]]
[[0.34862338]
 [0.83518601]
 [0.28969364]
 [0.17819426]
 [0.25544172]]
[[-0.58549436]
 [ 0.58644065]
 [ 0.09292856]
 [-0.55194928]]
[[0.34146967]
 [0.8383769 ]
 [0.29524377]
 [0.18028642]
 [0.24667904]]
[[-0.59211406]
 [ 0.58842632]
 [ 0.08511751]
 [-0.54397649]]
[[0.34448362]
 [0.84118186]
 [0.27796111]
 [0.172009  ]
 [0.25863998]]
[[-0.58263544]
 [ 0.59161126]
 [ 0.07784639]
 [-0.55178982]]
[[0.35079045]
 [0.83571596]
 [0.28221471]
 [0.17172611]
 [0.26341964]]
[[-0.

[[0.14699414]
 [0.8095465 ]
 [0.52224723]
 [0.12122066]
 [0.18865465]]
[[-0.53475286]
 [-0.19196795]
 [ 0.74290874]
 [-0.35394108]]
[[0.1422908 ]
 [0.80648689]
 [0.52167173]
 [0.12335263]
 [0.20487792]]
[[-0.53404515]
 [-0.18174656]
 [ 0.72151963]
 [-0.40146406]]
[[0.1496669 ]
 [0.80345747]
 [0.51712487]
 [0.13160606]
 [0.21752616]]
[[-0.53401387]
 [-0.16946513]
 [ 0.70469692]
 [-0.43533091]]
[[0.15716403]
 [0.80353602]
 [0.51417502]
 [0.14121366]
 [0.21286636]]
[[-0.56913759]
 [-0.17565188]
 [ 0.70582704]
 [-0.38345405]]
[[0.15298506]
 [0.80319302]
 [0.51356019]
 [0.1427831 ]
 [0.2175901 ]]
[[-0.5614358 ]
 [-0.16674299]
 [ 0.7017997 ]
 [-0.40554136]]
[[0.15620844]
 [0.80255604]
 [0.51076856]
 [0.14590977]
 [0.22210031]]
[[-0.56062815]
 [-0.15182203]
 [ 0.69405099]
 [-0.4253697 ]]
[[0.15546959]
 [0.80210562]
 [0.51195057]
 [0.13860957]
 [0.22616317]]
[[-0.53907647]
 [-0.13858409]
 [ 0.67779907]
 [-0.48039507]]
[[0.15619597]
 [0.80222771]
 [0.51384275]
 [0.13915505]
 [0.22053352]]
[[-0.

[[0.21327934]
 [0.75898348]
 [0.54954532]
 [0.18948217]
 [0.20137638]]
[[-0.61469548]
 [-0.02671033]
 [ 0.61212209]
 [-0.4967319 ]]
[[0.22567944]
 [0.75828604]
 [0.54288761]
 [0.19190584]
 [0.20619473]]
[[-0.63087554]
 [-0.03422502]
 [ 0.61674301]
 [-0.46952398]]
[[0.20849657]
 [0.75431858]
 [0.54813526]
 [0.21462301]
 [0.20252742]]
[[-0.64276292]
 [-0.02735023]
 [ 0.59821987]
 [-0.47774551]]
[[0.22273013]
 [0.75647089]
 [0.54432976]
 [0.19078227]
 [0.2131908 ]]
[[-0.6290086 ]
 [-0.03405173]
 [ 0.61348779]
 [-0.47625769]]
[[0.2070233 ]
 [0.75807938]
 [0.5461381 ]
 [0.20605217]
 [0.20428579]]
[[-0.65036627]
 [-0.0200278 ]
 [ 0.5950593 ]
 [-0.4717277 ]]
[[0.2216768 ]
 [0.75698248]
 [0.54929201]
 [0.19005086]
 [0.19998968]]
[[-0.63956892]
 [-0.02943086]
 [ 0.59692834]
 [-0.48348938]]
[[0.22819458]
 [0.75478073]
 [0.54628218]
 [0.19691867]
 [0.2025638 ]]
[[-0.6550594 ]
 [-0.02936159]
 [ 0.59585182]
 [-0.46367628]]
[[0.22824587]
 [0.75804183]
 [0.54871605]
 [0.18166483]
 [0.19795201]]
[[-0.

[[0.5286955 ]
 [0.61519199]
 [0.49000306]
 [0.22522694]
 [0.22625141]]
[[-0.24678658]
 [ 0.40974791]
 [ 0.7831358 ]
 [-0.39736804]]
[[0.50971361]
 [0.62796183]
 [0.50185783]
 [0.2121126 ]
 [0.22136607]]
[[-0.30008289]
 [ 0.32946094]
 [ 0.79883476]
 [-0.40406531]]
[[0.30449388]
 [0.67031273]
 [0.58305055]
 [0.23469406]
 [0.25086865]]
[[-0.81783511]
 [ 0.16435016]
 [ 0.49891121]
 [-0.23499439]]
[[0.26577979]
 [0.68053601]
 [0.59879166]
 [0.22390525]
 [0.23988918]]
[[-0.85376903]
 [ 0.19370208]
 [ 0.46483033]
 [-0.13225244]]
[[0.26590926]
 [0.6774515 ]
 [0.60052299]
 [0.2297048 ]
 [0.23866207]]
[[-0.85283376]
 [ 0.19187105]
 [ 0.47206314]
 [-0.11408976]]
[[0.26909865]
 [0.67373461]
 [0.59926276]
 [0.23787403]
 [0.24076479]]
[[-0.83437489]
 [ 0.18272648]
 [ 0.50547115]
 [-0.12218221]]
[[0.26741013]
 [0.6648018 ]
 [0.60513376]
 [0.24587042]
 [0.2447269 ]]
[[-0.80972491]
 [ 0.18302774]
 [ 0.53967409]
 [-0.13999393]]
[[0.26922   ]
 [0.66828504]
 [0.60777839]
 [0.24029915]
 [0.23189966]]
[[-0.

[[0.16505371]
 [0.54271138]
 [0.58220286]
 [0.51257653]
 [0.27663469]]
[[-0.92799261]
 [-0.09272998]
 [ 0.23357077]
 [ 0.27509191]]
[[0.19787238]
 [0.57801181]
 [0.58946458]
 [0.50446974]
 [0.15745049]]
[[-0.89397739]
 [-0.07903514]
 [ 0.2479195 ]
 [ 0.36482021]]
[[0.17843223]
 [0.60051583]
 [0.57293637]
 [0.49006448]
 [0.19779635]]
[[-0.89468902]
 [-0.11839259]
 [ 0.27097276]
 [ 0.33479623]]
[[0.18768739]
 [0.60816848]
 [0.58464206]
 [0.48310704]
 [0.14037729]]
[[-0.88090516]
 [-0.11049058]
 [ 0.25898031]
 [ 0.3804302 ]]
[[0.16295793]
 [0.61392967]
 [0.58311008]
 [0.49515346]
 [0.10649298]]
[[-0.86226022]
 [-0.12769835]
 [ 0.24537499]
 [ 0.42425413]]
[[0.16967887]
 [0.61817242]
 [0.58435477]
 [0.48704708]
 [0.10191461]]
[[-0.84819705]
 [-0.14234301]
 [ 0.22405021]
 [ 0.45836856]]
[[0.15394389]
 [0.63182185]
 [0.58266526]
 [0.48015746]
 [0.08397877]]
[[-0.79667294]
 [-0.16868946]
 [ 0.28206145]
 [ 0.50724494]]
[[0.19105867]
 [0.6450886 ]
 [0.58054914]
 [0.45022096]
 [0.08729874]]
[[-0.